In [2]:
import satpy
from pathlib import Path

In [3]:
SOURCE_PATH = Path("/mnt/storage/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/raw/EUMETSAT/SEVIRI_RSS/downloaded_manually_and_sorted/2019/06/01/12/04/MSG3-SEVI-MSG15-0100-NA-20190601120416.190000000Z-20190601120433-1358336-1.nat.bz2")